In [ ]:
# montar google drive para guardar y acceder a los archivos del proyecto
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Entrar a la carpeta del proyecto en Google Drive y mostrar todos los archivos que contiene
%cd "/content/drive/MyDrive/Pedro Yanez - neutral-farming-om"
!ls -R


/content/drive/MyDrive/Pedro Yanez - neutral-farming-om
.:
data  model  README.md	requirements.txt  src

./data:
organic_matter_dataset.csv

./model:
artifacts

./model/artifacts:
metrics.json  model.joblib

./src:
api.py	db.py  __init__.py  models.py  schema.py  train_model.py  utils.py


In [ ]:
# instalar las dependencias necesarias del proyecto desde requirements.txt
!pip install -r requirements.txt


In [ ]:
# ejecutar el script de entrenamiento para crear el modelo y guardar métricas en model/artifacts
!python src/train_model.py


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
Saved: /content/drive/MyDrive/Pedro Yanez - neutral-farming-om/model/artifacts/model.joblib
Metrics: {
  "mae": 0.13336367648681424,
  "rmse": 0.14653228960860765,
  "r2": 0.9561801798001207,
  "n_train": 8,
  "n_val": 2
}


In [ ]:
# abrir y mostrar las métricas de validación guardadas en model/artifacts/metrics.json
import json

with open("model/artifacts/metrics.json", "r") as f:
    metrics = json.load(f)

metrics


{'mae': 0.13336367648681424,
 'rmse': 0.14653228960860765,
 'r2': 0.9561801798001207,
 'n_train': 8,
 'n_val': 2}

In [ ]:
# instalar pyngrok para exponer la API de FastAPI al exterior con un link público
!pip install pyngrok


In [ ]:
# instalar cloudflared oficial
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!uvicorn src.api:app --host 0.0.0.0 --port 8000 & cloudflared tunnel --url http://localhost:8000


Selecting previously unselected package cloudflared.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.1) ...
Setting up cloudflared (2025.8.1) ...
Processing triggers for man-db (2.10.2-1) ...
2025-09-12T19:56:18Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-12T19:56:18Z INF Requesting new quick Tunnel on trycloudflare.com...
INFO:     Started serv

In [ ]:
# exponer la API en un link público sin password
!uvicorn src.api:app --host 0.0.0.0 --port 8000 & cloudflared tunnel --url http://localhost:8000

2025-09-12T20:08:41Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-12T20:08:41Z INF Requesting new quick Tunnel on trycloudflare.com...
INFO:     Started server process [20655]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2025-09-12T20:08:51Z INF +--------------------------------------------------------------------------------------------+
2025-

In [ ]:
#Ir a post predict try out y e indicar
# Link de ejemplo: https://pmc-tracks-aged-locations.trycloudflare.com/docs

{
  "pH": 6.7,
  "EC": 0.25,
  "Total_Nitrogen": 0.12,
  "Moisture": 28
}


In [ ]:
# endpoint /predict para construir un DataFrame con los nombres EXACTOS de entrenamiento
from pathlib import Path

api_code = """
from __future__ import annotations
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from .db import Base, engine, SessionLocal
from .models import SoilRecord
from .schema import IngestPayload, PredictPayload, PredictResponse
from .utils import get_model

Base.metadata.create_all(bind=engine)
app = FastAPI(title="Neutral Farming OM API", version="0.1.0")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.post('/data_ingestion')
def data_ingestion(payload: IngestPayload, db: Session = Depends(get_db)):
    record = SoilRecord(
        pH=payload.pH,
        EC=payload.EC,
        total_nitrogen=payload.Total_Nitrogen,
        moisture=payload.Moisture,
        organic_matter=payload.Organic_Matter,
    )
    db.add(record)
    db.commit()
    db.refresh(record)
    return {'id': record.id}

@app.post('/predict', response_model=PredictResponse)
def predict(payload: PredictPayload):
    import pandas as pd
    model = get_model()
    # construir DataFrame con los mismos nombres de columnas usados al entrenar
    X = pd.DataFrame([{
        'pH': payload.pH,
        'EC': payload.EC,
        'Total Nitrogen': payload.Total_Nitrogen,  # OJO: espacio como en el CSV
        'Moisture': payload.Moisture
    }])
    try:
        pred = float(model.predict(X)[0])
    except Exception as e:
        raise HTTPException(status_code=400, detail=f'Prediction failed: {e}')
    return PredictResponse(predicted_organic_matter=round(pred, 4))
"""
Path("src/api.py").write_text(api_code)
print("api.py actualizado 🎯")


api.py actualizado 🎯


In [ ]:
# Prueba de data ingestion
# Ir al link de ejemplo https://pmc-tracks-aged-locations.trycloudflare.com/docs
# Ir a POST /data_ingestion, try out, y usar el siguiente json de ejemplo

{
  "pH": 6.9,
  "EC": 0.28,
  "Total_Nitrogen": 0.14,
  "Moisture": 29,
  "Organic_Matter": 4.1
}


In [ ]:
# Configurar identidad de GitHub
!git config --global user.email "pedroeyanezmel531@gmail.com"
!git config --global user.name "p1y1m"

# Entrar a la carpeta del proyecto en Google Drive
%cd /content/drive/MyDrive/neutral-farming-om

# Inicializar git en la carpeta del proyecto
!git init

# Conectar el repositorio remoto de GitHub
!git remote add origin https://github.com/p1y1m/Codes.git

# Agregar todos los archivos del proyecto al versionamiento
!git add .

# Hacer el primer commit con un mensaje descriptivo
!git commit -m "Initial commit - Neutral Farming project"

# Cambiar el nombre de la rama a main
!git branch -M main

# Subir el proyecto a GitHub en la rama main
!git push -u origin main

